# Using Asteroid

This notebook was inspired by Andrew Shitov's excellent book [Using Raku: 100 Programming Challenges Solved with the Brand-New Raku Programming Language](https://andrewshitov.com/wp-content/uploads/2020/01/Using-Raku.pdf).  Here of course we use Asteroid to solve these programming challenges.

In [1]:
# make the Asteroid interpreter available in this notebook
from asteroid_interp import interp

**Note**: We use program as strings in this notebook so that we can run them right here in this notebook.

**Note**: There is a bug in the parser that does not allow for successive applications of member functions.  Those expressions need to be explicitly parenthesized.

# Part I
# Chapter 1: Strings
## 1.1 Using Strings


### 1. Hello, World!

> Print ‘Hello, World!’

The canonical `Hello, World!` program.  The easiest way to write this in Asteroid is,

In [2]:
program = \
'''
load "io".
println "Hello, World!".
'''
interp(program)

Hello, World!


Two other print functions exists: 
- `print` - print without appending a new line character.
- `raw_print` - print internal term structure. 

In [3]:
program = \
'''
load "io".
raw_print "Hello, World!".
'''
interp(program)

('string', 'Hello, World!')


Here we can see that an Asteroid string is tuple consisting of a type field and a value field.

### 2. Greet a person

> Ask a user for their name and greet them by printing ‘Hello, <Name\>!’

Here is our first solution using a separate function for each of the steps,   

In [4]:
program = \
'''
load "io".
print ("Enter your name: ").
let name = input().
print ("Hello, "+name+"!").
'''
interp(program)

Enter your name: Leo
Hello, Leo!

Letting the function `input` do the prompting,

In [5]:
program = \
'''
load "io".
let name = input("Enter your name: ").
print ("Hello, "+name+"!").
'''
interp(program)

Enter your name: Leo
Hello, Leo!

Doing everything in one step,

In [6]:
program = \
'''
load "io".
print ("Hello, "+input("Enter your name: ")+"!").
'''
interp(program)

Enter your name: Leo
Hello, Leo!

### 3. String length

> Print the length of a string.

In order to print the length of a string we can use the function `len` available in the `util` module,

In [7]:
program = \
'''
load "io".
load "util".
println (len("Hello!")).
'''
interp(program)

6


We can also use the string member function `length` in order to compute the length of the string,

In [8]:
program = \
'''
load "io".
println ("Hello!" @length()).
'''
interp(program)

6


### 4. Unique digits

> Print unique digits from a given integer number.

In order to accomplish this we take advantage of the string `explode` function and the `sort` function on lists.
Finally we use the `reduce` function to map a list with repeated digits to a list with unique digits,

In [9]:
program = \
'''
load "io".

function unique with (x,y) do
    if not (x @member(y)) do
        return x @append(y).
    else do
        return x.
    end
end
        
let digits = (("332211" @explode()) @sort()) @reduce(unique,[]).
println digits.
assert(digits == ["1","2","3"]).
'''
interp(program)

[1,2,3]


Probably the most noteworthy characteric about this program is the `reduce` function.  The `reduce` function applies a binary function to a list.  The first argument of the binary function acts like an accumulator and the second argument get instantiated with the elements of the list to be processed.  In our function `unique` the variable `x` is the accumulator with an initial value of `[]`.  The function tests whether the element `y` is on the list.  If it is not then it adds it to the list otherwise it just returns the accumulator unchanged.

## 1.2 Modifying string data

### 5. Reverse a string

> Print a string in the reversed order from right to left.

We use the `explode` function to turn a string into a list of characters, then we reverse the list and turn it back into a string using the `join` function,

In [10]:
program = \
'''
load "io".
let str = (("Hello, World!" @explode()) @reverse()) @join("").
println str.
assert(str == "!dlroW ,olleH").
'''
interp(program)

!dlroW ,olleH


### 6. Removing blanks from a string

> Remove leading, trailing and double spaces from a given string.

In [11]:
program = \
'''
load "io".
let str = ("   Hello  ,   World    !   " @trim()) @replace("  ","").
println str.
assert(str == "Hello, World!").
'''
interp(program)

Hello, World!


### 7. Camel case

> Create a camel-case identifier from a given phrase.

In this task, we will form the `CamelCase` variable names from a given phrase.
Names created in this style are built of several words; each of which starts
with a capital letter.

In [12]:
program = \
'''
load "io".

function title with w do
    let letter_list = (w @tolower()) @explode().
    let first_letter = (letter_list@0) @toupper().
    if letter_list @length() > 1 do
        let title_case = ([first_letter]+letter_list@[1 to letter_list@length()-1]) @join("").
    else
        let title_case = first_letter.
    end
    return title_case.
end

let str = "once upon a time".
let camel_str = ((str @split()) @map(title)) @join("").
println camel_str.
assert(camel_str == "OnceUponATime").
'''
interp(program)

OnceUponATime


### 8. Incrementing filenames

> Generate a list of filenames like file1.txt, file2.txt, etc.

In [13]:
program = \
'''
load "io".

let root = "file".
let ext = ".txt".

for i in 1 to 5 do
    println (root+i+ext).
end
'''
interp(program)

file1.txt
file2.txt
file3.txt
file4.txt
file5.txt


### 9. Random passwords

> Generate a random string that can be used as a password.

In our solution we take advantage of Asteroid's `Pick` object.  The `Pick` object maintains a list of items that we can randomly select from using the `pick` member function.  As input to the `Pick` object we compute a bunch of lists of characters that are useful for password construction.  The function `achar` converts a decimal ASCII code to a single character string.

In [14]:
program = \
'''
load "io".
load "util".

-- make up lists of symbols useful for password construction
let int_list = [0 to 9] @map(tostring).
let lc_list = [97 to 122] @map(achar). -- lower case characters
let uc_list = [65 to 90] @map(achar). --upper case characters
let sp_list = ["!","_","#","$","%","*"].
-- build the overall pick list of symbols
let pick_list = int_list+lc_list+uc_list+sp_list.

-- generate the password and print it.
println ((Pick pick_list) @pick(15)) @join("").
'''
interp(program)

OQJrtGB##dNjR9$


### 10. DNA-to-RNA transcription

> Convert the given DNA sequence to a compliment RNA.

We’ll not dig deep into the biology aspect of the problem. For us, it is important that the DNA is a string containing the four letters A, C, G, and T,
and the RNA is a string of A, C, G, and U. The transformation from DNA
to RNA happens according to the following table:
```
DNA: A C G T
RNA: U G C A
```
We will solve this programming problem using Asteroid's first-class patterns. We could have solved this with just testing equality on DNA characters but using first-class patterns in more general and can be applied to problems with a more structured mapping relationship.

In [15]:
program = \
'''
load "io".
load "util".

let dna2rna_table = 
    [
    ("A","U"),
    ("C","G"),
    ("G","C"),
    ("T","A")
    ].

function dna2rna with x do
    for (dna,rna) in dna2rna_table do
        if x is *dna do
            return rna.
        end
    end
    throw Error("unknown dna char "+x).
end

let dna_seq = "ACCATCAGTC".
let rna_seq = ((dna_seq @explode()) @map(dna2rna)) @join("").
println rna_seq.
assert(rna_seq == "UGGUAGUCAG").
'''
interp(program)

UGGUAGUCAG


### 11. Caesar cipher

> Encode a message using the Caesar cipher technique.

The Caesar code is a simple method of transcoding the letters of the message
so that each letter is replaced with the letter that occurs in the alphabet N
positions earlier or later.
For example, if N is 4, then the letter e becomes a, f is transformed to b,
etc. The alphabet is looped so that z becomes v, and letters a to d become
w to z.

In [16]:
program = \
'''
load "io".
load "util".

let encode_table = [119 to 122] @map(achar) + [97 to 118] @map(achar).

function encode with (v:%string) %if len(v) == 1 do
    -- only lowercase letters are encoded
    if not (ascii(v) in [97 to 122]) do
        return v.
    else 
        return encode_table @(ascii(v)-ascii("a")).
    end
end

function decode with (v:%string) %if len(v) == 1 do
    -- only lowercase letters are decoded
    if not (ascii(v) in [97 to 122]) do
        return v.
    else 
        return encode_table @(ascii(v)-ascii("w")+4).
    end
end

let message = "hello, world!"
let secret = ((message @explode()) @map(encode)) @join("").
println secret.

let decoded_msg = ((secret @explode()) @map(decode)) @join("").
println decoded_msg.
'''
interp(program)

dahhk, sknhz!
hello, world!


## 1.3 Text Analysis

### 12. Plural Endings

> Put a noun in the correct form — singular or plural — depending on the number next to it.

In program outputs, it is often required to print some number followed by a noun, for example:
```
10 files found
```
If there is only one file, then the phrase should be `1 file found` instead.

In [17]:
program = \
'''
load "io".

for n in 0 to 5 do
    println (n+" file"+("s " if n>1 or n==0 else " ")+"found").
end
'''
interp(program)

0 files found
1 file found
2 files found
3 files found
4 files found
5 files found


### 13. The most frequent word

> Find the most frequent word in the given text.


In [18]:
program = \
'''
load "io".
load "util".

-- text generated at 'https://www.lipsum.com/'
let text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed accumsan magna quis risus commodo, et pellentesque dui cursus. Sed quis risus libero. Cras et mattis libero, eget varius nisi. Phasellus ultrices, augue non dictum eleifend, nunc elit blandit velit, a viverra risus enim in tellus. Maecenas quis ante eget turpis rhoncus rhoncus eget ut mauris. Suspendisse nec erat sed nunc tempus hendrerit. Nunc dictum nunc molestie eleifend tempus. Praesent cursus lorem diam, sed mattis velit vehicula scelerisque. Nunc iaculis rhoncus ante. Etiam quam nisi, fermentum et euismod a, vulputate eu elit. Suspendisse tincidunt ligula quis interdum blandit. Quisque sed aliquam tellus. Pellentesque ac lacus pulvinar, ornare purus ac, viverra ex. Donec quis pharetra dolor.

In ac massa tortor. Cras sagittis luctus scelerisque. Morbi a neque sed tortor ultrices dapibus. Mauris pretium vitae massa non auctor. Cras egestas ex ante, ac ullamcorper ante dignissim eget. Fusce bibendum justo ut enim luctus, id volutpat diam lacinia. Mauris sit amet ante risus.

Nullam rhoncus ultricies dui. Etiam vel metus vehicula, pellentesque felis ut, suscipit nunc. Sed nec interdum lorem. Maecenas odio erat, vestibulum nec dapibus id, commodo vitae libero. Nulla sed urna sit amet nunc commodo finibus sed vel elit. Aliquam euismod feugiat nisi quis placerat. Aliquam libero nisl, ultrices non est at, sagittis hendrerit dui. Quisque id sem lorem. Nam ultricies metus id ultrices molestie. Pellentesque elementum consequat nibh, nec convallis lorem ullamcorper in. Etiam vitae mi tellus. Etiam accumsan massa sit amet dolor tincidunt iaculis. Nam ullamcorper blandit sem id bibendum. Quisque elementum ipsum ac sapien blandit vehicula."

-- get rid of punctuation, turn to lower case, and split into words.
-- Note: we could have employed richer regular expressions to clean up the text here
let wl = (((text @replace("\.","")) @replace(",","")) @tolower()) @split().

-- put the words into a hash table, the value is the count of the words
let ht = HashTable().
for w in wl do
    if not ht @get(w) do
        ht @insert(w,1).
    else do
        ht @insert(w,ht @get(w)+1).
    end
end

-- get the contents of hash table and find the most frequent word
let (keys,values) = unzip(ht @aslist()).
let values_sorted = (values @copy()) @sort(true).
let most_frequent_word = keys @(values @index(values_sorted @0)).
println most_frequent_word.
assert (most_frequent_word == "sed").
'''
interp(program)

sed


### 14. The longest common substring

> Find the longest common substring in the given two strings.

Let us limit ourselves with finding only the first longest substring. If there
are more common substrings of the same length, then the rest are ignored.
There are two loops (see also Task 17, The longest palindrome) over the first
string (`stra`), and they use the index method to search for the substring in the
second string (`strb`).

In [19]:
program = \
'''
load "io".
load "util".

let stra = "the quick brown fox jumps over the lazy dog".
let strb = "what does the fox say?".
let common = "".

for startix in 0 to stra @length()-1 do
    for endix in startix to stra @length()-1 do
        let s = stra @[startix to endix].
        if strb @index(s) and s @length() > common @length() do
            let common = s.
        end
    end
end

if common do
    println ("The longest common substring is '"+common+"'.").
else do
    println ("There are no common substrings.").
end

assert (common == " fox ").
'''
interp(program)

The longest common substring is ' fox '.


### 15. Anagram test

> Tell if the two words are anagrams of each other.

An anagram is a word, phrase, or name formed by rearranging the letters of another, such as `cinema`, formed from `iceman`.

In [20]:
program = \
'''
load "io".

let str1 = "cinema".
let str2 = "iceman".

function normalize with str do
    return ((str @explode()) @sort()) @join("").
end

if normalize(str1) == normalize(str2) do
    println "Anagrams".
else do
    println "Not anagrams".
end

assert (normalize(str1) == normalize(str2)).
'''
interp(program)

Anagrams


### 16. Palindrome test

> Check if the entered string is palindromic.

A palindrome is a string that can be read from both ends: left to right or right
to left.

In [21]:
program = \
'''
load "io".

let str = "Was it a rat I saw?".

function clean with str:%string do
    return (str @tolower()) @replace("[^a-z]","").
end

-- only keep lower case letters
let clean_str = clean(str).

-- check if it is palidromic
if clean_str == clean_str @flip() do
    println "Palindromic".
else do
    println "Not palindromic".
end

assert (clean_str == clean_str @flip()).
'''
interp(program)

Palindromic


### 17. The longest palindrome

> Find the longest palindromic substring in the given string.

The main idea behind the solution is to scan the string with a window of
varying width. In other words, starting from a given character, test all the
substrings of any length possible at that position. 
Now, extract the substring and do the check similar to the solution of Task
16, Palindrome test. Here, we have to be careful to check the palindrome
without taking into account the non-letter characters but saving the result as
part of the original string. 

In [23]:
program = \
'''
load "io".

let str = "Hello, World!".

function clean with str:%string do
    return (str @tolower()) @replace("[^a-z]","").
end

function palindrome_test with str:%string do
    let clean_str = clean(str).
    if clean_str == clean_str @flip() do
        return true.
    else do
        return false.
    end
end

-- create the moving window over the string
let longest_palindrome = "".

for i in 0 to str @length()-2 do
    for j in i+1 to str @length()-1 do
        let str1 = str @[i to j].
        if palindrome_test(str1) and 
           str1 @length() > longest_palindrome @length() do 
            let longest_palindrome = str1.
        end
    end
end

println longest_palindrome.

'''
interp(program)

o, Wo


### 18. Finding duplicate texts

> Find duplicate fragments in the same text.

We do this by finding and hashing N-grams after the appropriate preprocessing.  We will use `N=3`.

In [34]:
program = \
'''
load "io".
load "util".

-- text from "www.lipsum.com"

let str = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed malesuada sapien nec neque suscipit, non rutrum arcu scelerisque. Nam feugiat sapien porta ipsum accumsan, eget maximus diam volutpat. Pellentesque elementum in orci quis pretium. Donec dignissim nunc lectus, id ornare urna varius ut. Praesent semper faucibus vehicula. Aliquam luctus sapien at lorem malesuada, eget suscipit felis facilisis. Suspendisse velit lectus, mollis sit amet tempor eget, faucibus ut nulla. Vestibulum et elementum dolor, a vehicula ipsum. Morbi ut fringilla nisi. Fusce congue rutrum orci nec porta. Ut laoreet justo vel turpis sodales vehicula. Nulla porttitor nisl id odio eleifend sodales.

Suspendisse blandit tristique enim id laoreet. Etiam vel aliquet dui, quis tempus magna. Donec blandit volutpat felis egestas tincidunt. Integer placerat luctus mi non pharetra. Donec aliquet nisl orci, egestas elementum nunc bibendum a. Morbi nec risus aliquet, viverra nunc in, molestie odio. Curabitur pellentesque, ante eget dictum aliquam, felis leo bibendum libero, vel bibendum lorem velit eget ex. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vestibulum pretium tellus quis ante vulputate, pretium tincidunt ipsum dapibus. Praesent congue, ipsum ut sagittis tempus, lacus nisi dapibus dui, aliquam porta metus odio ut neque. Aliquam vitae faucibus dolor. Nulla iaculis lorem non mauris viverra, ut malesuada nibh aliquam. Nam bibendum sit amet massa in dignissim. Nam posuere nunc ante, at viverra diam rhoncus vel.

Aliquam mollis sagittis nulla. Maecenas faucibus eu dui eget accumsan. Suspendisse sit amet fermentum sapien. Nunc vitae mi nibh. Mauris condimentum vestibulum imperdiet. Quisque at vehicula dui. Integer sit amet volutpat arcu. Maecenas efficitur leo tortor, non ullamcorper magna tempor non. Sed efficitur quis metus ut pulvinar. Proin nunc felis, congue sit amet nibh placerat, tincidunt mattis nunc. Duis efficitur lacus a orci porttitor, sed molestie risus tempor.

Sed tincidunt ipsum at urna sollicitudin feugiat. Ut mollis orci quis massa dictum facilisis. Maecenas non elementum mauris. Sed rutrum orci faucibus, tristique nunc nec, mattis ante. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. In hac habitasse platea dictumst. Morbi pellentesque dolor sit amet nunc tincidunt, ut rutrum ante vulputate. Nullam pretium, mi sed condimentum luctus, ipsum nunc dictum lorem, vel ultricies nibh mi ut sem. Nam volutpat id libero eget mollis.

Vestibulum eget velit eros. Phasellus sit amet vestibulum odio, vel malesuada quam. Mauris dictum erat eu ligula mollis laoreet. Phasellus ut ante auctor, hendrerit ipsum et, fermentum magna. Etiam nec eros elementum, consectetur nibh ac, ullamcorper ligula. Aliquam sed porttitor sapien. Nulla tincidunt, turpis vitae venenatis aliquet, quam purus elementum diam, in tincidunt orci diam sed nulla. Cras pellentesque non diam quis sollicitudin. Duis suscipit lectus dui, eu varius metus pretium sit amet.

Nulla eu ex velit. Ut non justo semper, gravida erat quis, vehicula est. Suspendisse nunc dui, iaculis id purus sit amet, rutrum commodo lacus. Aenean consequat turpis a est vestibulum, ac accumsan nibh dapibus. Nam blandit scelerisque lectus, eu pellentesque arcu ornare non. Fusce ac gravida diam. Ut in fringilla eros. Sed metus augue, porta quis vehicula at, pellentesque et mauris. Duis sodales lacus sit amet condimentum placerat. In blandit tristique nulla eget malesuada. Sed congue finibus neque at semper. Etiam pellentesque egestas urna, ut lobortis odio euismod et. Phasellus aliquet quam purus, quis ullamcorper sem mollis eu.

Mauris quis ullamcorper nisi. Aenean quam nulla, sodales eu faucibus in, mattis a nulla. Nullam pulvinar pretium justo eu mattis. Aliquam rutrum ipsum vitae leo maximus ultrices. Donec ut pulvinar nisi. Sed pharetra, turpis dictum lobortis egestas, quam massa venenatis enim, dapibus efficitur dolor mauris eu felis. Donec vulputate ultrices justo sit amet condimentum. Donec id posuere nulla. In vestibulum mi in lectus commodo dignissim. Quisque vestibulum egestas arcu sit amet finibus. Proin commodo aliquet neque quis maximus.

Nulla facilisi. Sed gravida aliquet diam in congue. Mauris vehicula justo ac sollicitudin laoreet. Mauris enim mi, auctor id magna eget, feugiat sollicitudin leo. Vivamus ornare ornare commodo. Suspendisse ut dui quis enim porta pretium. Praesent vitae lacus fermentum, posuere orci ac, imperdiet massa. Nulla hendrerit id nisl sed maximus. Vivamus commodo lacus eu condimentum bibendum. Suspendisse porttitor sem eget dolor aliquet congue. Pellentesque tristique augue at quam hendrerit dignissim. Aenean a congue dui. Vestibulum ante ipsum primis in faucibus orci luctus et ultrices posuere cubilia curae; Integer ante lacus, commodo et enim sed, auctor egestas metus.

Aliquam a urna id risus tincidunt rutrum. Nunc facilisis, tortor ac suscipit aliquam, ante neque tincidunt mi, nec ullamcorper lectus ligula vel urna. Suspendisse lobortis at felis sit amet facilisis. Pellentesque velit lacus, porttitor vitae eros rutrum, convallis blandit erat. Pellentesque nec mi viverra, volutpat dui in, rutrum lacus. Ut non venenatis leo. Praesent sollicitudin magna porttitor lorem elementum molestie non a turpis. Suspendisse potenti.

Donec malesuada iaculis laoreet. Nunc ut volutpat ante, ut consequat tortor. Phasellus posuere, ipsum quis dignissim iaculis, nisl felis ullamcorper ligula, quis placerat sem sapien nec ante. Cras suscipit ut magna nec lacinia. Donec ipsum nibh, imperdiet non aliquam eu, maximus id ante. Pellentesque vitae felis felis. Aliquam et diam sed nulla volutpat vestibulum molestie non lacus. Praesent porta et lacus auctor fermentum. In hac habitasse platea dictumst. Aliquam erat volutpat. Etiam at ligula orci. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos."


let word_list = ((str @tolower()) @replace("[^a-z0-9_]"," ")) @split().
let ht = HashTable().

-- create N-grams
for i in 0 to word_list @length()-3 do
    let n_gram = [word_list@i, word_list@(i+1), word_list @(i+2)] @join(" ").
    -- put the N-gram into a hash table, the value is the count of the N-gram in the text.
    if not ht @get(n_gram) do
        ht @insert(n_gram,1).
    else do
        ht @insert(n_gram,ht @get(n_gram)+1).
    end
end

for ((n_gram,cnt) %if cnt > 1) in ht @aslist() do
    println (n_gram+": "+cnt).
end
'''
interp(program)

lorem ipsum dolor: 2
ipsum dolor sit: 2
dolor sit amet: 3
sit amet consectetur: 2
amet consectetur adipiscing: 2
consectetur adipiscing elit: 2
in hac habitasse: 2
hac habitasse platea: 2
habitasse platea dictumst: 2
aliquet quam purus: 2
diam sed nulla: 2
sit amet condimentum: 2
